In [1]:
import pandas as pd

%reload_ext autoreload
%autoreload 2

# # Tell python where to look for modules.
import sys

sys.path.append("../../hourly-egrid/")
from src.column_checks import get_dtypes
import src.load_data as load_data

# NOX

When calculating NOx:
Apply emission rates in the following order based on availability:  
    1. boiler-specific ozone-season nox emission rates reported in eia923 schedule 8C  
    2. boiler-specific annual nox emission rates reported in eia923 scehdule 8C  
    3. fuel, prime mover, and boiler firing type specific emission factors  

Control-specific emission rates:
- [x] Load control-specific nox rates from EIA-923 8C
- [x] Load control-boiler associations
- [ ] Load control attributes
- [ ] Map controls to boilers
- [ ] Map boilers to generators 
- [ ] Calculate generator-specific emission rates

Boiler firing types

In [2]:
year = 2020
eia_filepath = "../../../EIA Data/"

emissions_controls_eia923_names = [
    "report_date",
    "plant_id_eia",
    "equipment_tech_description",
    "pm_control_id",
    "so2_control_id",
    "nox_control_id",
    "mercury_control_id",
    "operational_status",
    "hours_in_service",
    "annual_nox_emission_rate_lb_per_mmbtu",
    "ozone_season_nox_emission_rate_lb_per_mmbtu",
    "pm_emission_rate_lb_per_mmbtu",
    "pm_removal_efficiency_annual",
    "pm_removal_efficiency_at_full_load",
    "pm_test_date",
    "so2_removal_efficiency_annual",
    "so2_removal_efficiency_at_full_load",
    "so2_test_date",
    "fgd_sorbent_consumption_1000_tons",
    "fgd_electricity_consumption_mwh",
    "hg_removal_efficiency",
    "hg_emission_rate_lb_per_trillion_btu",
    "acid_gas_removal_efficiency",
]

emissions_controls_eia923 = pd.read_excel(
    io=(
        eia_filepath
        + f"f923_{year}/EIA923_Schedule_8_Annual_Environmental_Information_{year}_Final_Revision.xlsx"
    ),
    sheet_name="8C Air Emissions Control Info",
    header=4,
    names=emissions_controls_eia923_names,
    dtype=get_dtypes(),
    na_values=".",
    parse_dates=["report_date","pm_test_date","so2_test_date"]
)

emissions_controls_eia923

,report_date,plant_id_eia,equipment_tech_description,pm_control_id,so2_control_id,nox_control_id,mercury_control_id,operational_status,hours_in_service,annual_nox_emission_rate_lb_per_mmbtu,ozone_season_nox_emission_rate_lb_per_mmbtu,pm_emission_rate_lb_per_mmbtu,pm_removal_efficiency_annual,pm_removal_efficiency_at_full_load,pm_test_date,so2_removal_efficiency_annual,so2_removal_efficiency_at_full_load,so2_test_date,fgd_sorbent_consumption_1000_tons,fgd_electricity_consumption_mwh,hg_removal_efficiency,hg_emission_rate_lb_per_trillion_btu,acid_gas_removal_efficiency
0,2020-01-01,3,Activated carbon injection system,NaN,NaN,NaN,4,OP,798.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.45,NaN
1,2020-01-01,3,Dry sorbent (powder) injection type (DSI),NaN,NaN,NaN,NaN,OP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-01-01,3,"Electrostatic precipitator, cold side, with fl...",5,NaN,NaN,5,OP,6464.0,NaN,NaN,0.0093,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.36,NaN
3,2020-01-01,3,"Electrostatic precipitator, cold side, with fl...",4,NaN,NaN,4,OP,916.0,NaN,NaN,0.0240,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.45,NaN
4,2020-01-01,3,"Electrostatic precipitator, hot side, without ...",2,NaN,NaN,NaN,OP,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7095,2020-01-01,60927,Selective catalytic reduction,NaN,NaN,1BSCR,NaN,OP,1938.0,0.014,0.007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7096,2020-01-01,60927,Selective catalytic reduction,NaN,NaN,1ASCR,NaN,OP,2104.0,0.014,0.008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7097,2020-01-01,61028,Selective catalytic reduction,NaN,NaN,SCR-1,NaN,OP,5731.0,0.007,0.007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7098,2020-01-01,61028,Selective catalytic reduction,NaN,NaN,SCR-2,NaN,OP,5619.0,0.008,0.008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
boiler_nox_association_eia860_names = [
    "utility_id_eia",
    "utility_name_eia",
    "plant_id_eia",
    "plant_name_eia",
    "boiler_id",
    "nox_control_id",
    "steam_plant_type",
]

boiler_nox_association_eia860 = pd.read_excel(
    io=(
        eia_filepath
        + f"eia860{year}/6_1_EnviroAssoc_Y{year}.xlsx"
    ),
    sheet_name="Boiler NOx",
    header=1,
    names=boiler_nox_association_eia860_names,
    dtype=get_dtypes(),
    na_values=".",
    skipfooter=1
)

boiler_nox_association_eia860

,utility_id_eia,utility_name_eia,plant_id_eia,plant_name_eia,boiler_id,nox_control_id,steam_plant_type
0,195,Alabama Power Co,3,Barry,1,1,1
1,195,Alabama Power Co,3,Barry,2,2,1
2,195,Alabama Power Co,3,Barry,3,3,1
3,195,Alabama Power Co,3,Barry,4,4,1
4,195,Alabama Power Co,3,Barry,5,5,1
...,...,...,...,...,...,...,...
1763,61001,"Hu Honua Bioenergy, LLC",61364,Hu Honua Bioenergy Facility,23523,1,2
1764,5109,DTE Electric Company,62192,Blue Water Energy Center,1,1S,1
1765,62806,Guernsey Power Station LLC,62949,Guernsey Power Station,HRSG1,SCR1,1
1766,62806,Guernsey Power Station LLC,62949,Guernsey Power Station,HRSG2,SCR2,1


In [4]:
pudl_out = load_data.initialize_pudl_out(year)
bga = pudl_out.bga_eia860()

### Combine data

In [5]:
nox_rates = emissions_controls_eia923[["plant_id_eia","nox_control_id","annual_nox_emission_rate_lb_per_mmbtu","ozone_season_nox_emission_rate_lb_per_mmbtu",]].dropna(thresh=2)
nox_rates

,plant_id_eia,nox_control_id,annual_nox_emission_rate_lb_per_mmbtu,ozone_season_nox_emission_rate_lb_per_mmbtu
7,3,5,0.079,0.073
8,3,1,0.052,NaN
9,3,2,0.052,NaN
10,3,7B,0.009,0.008
11,3,7A,0.009,0.008
...,...,...,...,...
7095,60927,1BSCR,0.014,0.007
7096,60927,1ASCR,0.014,0.008
7097,61028,SCR-1,0.007,0.007
7098,61028,SCR-2,0.008,0.008


In [10]:
boiler_nox_association_eia860.loc[boiler_nox_association_eia860[["plant_id_eia","nox_control_id"]].duplicated(keep=False), ["plant_id_eia","nox_control_id","boiler_id",]]

,plant_id_eia,nox_control_id,boiler_id
70,384,7,71
71,384,7,72
72,384,8,82
73,384,8,81
159,667,2,2
...,...,...,...
1725,60122,LN SR,HRSG7
1731,60356,SCR,HRSG2
1732,60356,SCR,HRSG1
1740,60464,ULNB,HRSG2


In [8]:
nox_rates.merge(boiler_nox_association_eia860[["plant_id_eia","nox_control_id","boiler_id",]], how="left", on=["plant_id_eia","nox_control_id"])

,plant_id_eia,nox_control_id,annual_nox_emission_rate_lb_per_mmbtu,ozone_season_nox_emission_rate_lb_per_mmbtu,boiler_id
0,3,5,0.079,0.073,5
1,3,1,0.052,NaN,1
2,3,2,0.052,NaN,2
3,3,7B,0.009,0.008,7B
4,3,7A,0.009,0.008,7A
...,...,...,...,...,...
4428,60927,1BSCR,0.014,0.007,1BHRSG
4429,60927,1ASCR,0.014,0.008,1AHRSG
4430,61028,SCR-1,0.007,0.007,HRSG
4431,61028,SCR-2,0.008,0.008,HRSG-2
